In [10]:
from transformers import BertModel, BertForSequenceClassification

In [11]:
model_name = 'bert-base-uncased'

In [14]:
#model = BertModel.from_pretrained(model_name)
model_cls = BertForSequenceClassification.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model_cls

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [13]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [77]:
for name, params in model.named_parameters():
    if 'encode' in name:
        print(name, params)
        break

encoder.layer.0.attention.self.query.weight Parameter containing:
tensor([[-0.0164,  0.0261, -0.0263,  ...,  0.0154,  0.0768,  0.0548],
        [-0.0326,  0.0346, -0.0423,  ..., -0.0527,  0.1393,  0.0078],
        [ 0.0105,  0.0334,  0.0109,  ..., -0.0279,  0.0258, -0.0468],
        ...,
        [-0.0085,  0.0514,  0.0555,  ...,  0.0282,  0.0543, -0.0541],
        [-0.0198,  0.0944,  0.0617,  ..., -0.1042,  0.0601,  0.0470],
        [ 0.0015, -0.0952,  0.0099,  ..., -0.0191, -0.0508, -0.0085]],
       requires_grad=True)


## Bert Model Architecture
- Bert: encoder of transformer
  - transformer: encoder-decoder (seq2seq)
- embedding
  - wording_embedding
  - position_embedding
  - token_type_embedding
- encoder (12 layers)
  - self-attention: query, key, value (qkv)
  - feed-forward
- pooler

In [91]:
# stats on parameters
total_params = 0
total_learnable_params = 0
total_embedding_params = 0
total_encode_params = 0

layer_encode_params = {}
total_pooler_params = 0
temp = 0


for name, params in model.named_parameters():
    #print(name, '->', params.shape, '->', params.numel())
    if 'embedding' in name: 
        total_embedding_params +=params.numel()
    if 'encode' in name: 
        total_encode_params +=params.numel()

        layer_id = name.split('.')[2]
        
        if layer_id not in layer_encode_params: 
            layer_encode_params[layer_id] = 0

        layer_encode_params[layer_id] += params.numel()
        
    if 'pooler' in name: 
        total_pooler_params +=params.numel()
    
    if params.requires_grad:
        total_learnable_params +=params.numel()
     
    total_params += params.numel()



In [80]:
print(total_embedding_params, total_encode_params, total_pooler_params)
print(total_learnable_params, total_params)

23837184 85054464 590592
109482240 109482240


In [81]:
plist = [total_embedding_params, total_encode_params, total_pooler_params]
[i*100/sum(plist) for i in plist]



[21.772649152958508, 77.68790992950089, 0.5394409175405984]

In [92]:
layer_encode_params

{'0': 7087872,
 '1': 7087872,
 '2': 7087872,
 '3': 7087872,
 '4': 7087872,
 '5': 7087872,
 '6': 7087872,
 '7': 7087872,
 '8': 7087872,
 '9': 7087872,
 '10': 7087872,
 '11': 7087872}

In [93]:
for name, params in model.named_parameters():
    #print(name, '->', params.shape, '->', params.numel())
    if 'encode' in name:
        print(name)

encoder.layer.0.attention.self.query.weight
encoder.layer.0.attention.self.query.bias
encoder.layer.0.attention.self.key.weight
encoder.layer.0.attention.self.key.bias
encoder.layer.0.attention.self.value.weight
encoder.layer.0.attention.self.value.bias
encoder.layer.0.attention.output.dense.weight
encoder.layer.0.attention.output.dense.bias
encoder.layer.0.attention.output.LayerNorm.weight
encoder.layer.0.attention.output.LayerNorm.bias
encoder.layer.0.intermediate.dense.weight
encoder.layer.0.intermediate.dense.bias
encoder.layer.0.output.dense.weight
encoder.layer.0.output.dense.bias
encoder.layer.0.output.LayerNorm.weight
encoder.layer.0.output.LayerNorm.bias
encoder.layer.1.attention.self.query.weight
encoder.layer.1.attention.self.query.bias
encoder.layer.1.attention.self.key.weight
encoder.layer.1.attention.self.key.bias
encoder.layer.1.attention.self.value.weight
encoder.layer.1.attention.self.value.bias
encoder.layer.1.attention.output.dense.weight
encoder.layer.1.attention.ou